In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import kagglehub
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer
from tqdm import tqdm
import sklearn
from sklearn import metrics

from dataclasses import dataclass, field
from typing import Optional, List

### Initialize LLM

Start Ollama Docker container (see Resources/docker.sh) and call the model specified in the model_id

In [ ]:
@dataclass
class ModelInfo:
    model_path: str
    model_description: str
    model: Optional[object] = None
    tokenizer: Optional[object] = None

@dataclass
class ModelRegistry:
    models: List[ModelInfo] = field(default_factory=list)
    current_index: int = 0

    def add_model(self, model_path: str, model_description: str):
        self.models.append(ModelInfo(model_path, model_description))

    def load_model(self, index: int):
        info = self.models[index]
        info.model = AutoModelForCausalLM.from_pretrained(info.model_path)
        info.tokenizer = AutoTokenizer.from_pretrained(info.model_path)
        self.current_index = index
        return info.model, info.tokenizer, info.model_path

    def get_current_model(self):
        return self.models[self.current_index]
    
# initialize models in registry

model_registry = ModelRegistry()
model_registry.add_model("google/gemma-3-4b-it", "Gemma 3B Instruction-Tuned")
model_registry.add_model("meta-llama/Llama-2-7b-hf", "Llama 2 7B HF")



In [ ]:
# download tokenizer
model, tokenizer, model_path = model_registry.load_model(0)

In [ ]:
# choose pipeline criteria
task = "text-generation"
weights = {"torch_dtype": torch.bfloat16} # quantization = 16, 32, 64

# initate pipeline
pipeline = transformers.pipeline(task, model = model_path, model_kwargs = weights) # initate pipeline

### Read Data
Data are from the Kaggle Competition, Jigsaw - Agile Community Rules Classification.

Save the following files to the data folder:
- train.csv
- test.csv
- sample_submission.csv

In [ ]:
df = pd.read_csv("data/train.csv")

### Generate Prompts
Create a loop that first uses information from df to generate a prompt, supply it to the LLM, and then extract the output into the probability list.

In [ ]:
# create prompt loop
prompts = []

for each_row in df.index:
    rule = df["rule"][each_row]
    body = df["body"][each_row]
    input = "Calculate the probability that a sample of text violates a rule where, 0 = rule is not violated to 1 = rule is violated. Here is the rule: " + rule + ". Here is the text: " + body + ". Structure your response as only a numeric probability ranging from 0.0 to 1.0."
    prompts.append(input)
    

### Prompt the Model

Here's an example of the prompting system but with a single case:

In [ ]:
out_tokens = 10 # n characters in output

# generate responses
responses = []
for each_prompt in tqdm(prompts):
    input = [{"role": "user", "content": each_prompt}]
    output = pipeline(input, max_new_tokens = out_tokens)
    new_response = output[0]["generated_text"][-1]["content"]
    responses.append(new_response)
    

In [ ]:
prompt = "Calculate the probability that a sample of text violates a rule where, 0 = rule is not violated to 1 = rule is violated. Here is the rule: " + df["rule"][0] + ". Here is the text: " + df["body"][0] + ". Structure your response as only a numeric probability ranging from 0.0 to 1.0."
input = [{"role": "user", "content": prompt}]
output = pipeline(input, max_new_tokens = out_tokens)
response = output[0]["generated_text"][-1]["content"]
response

### Evaluate Responses
Evaluate the responses based on the area under the curve (AUC)

In [ ]:
y_true = df["rule_violation"]
y_obs = responses
fpr, tpr, thresholds = metrics.roc_curve(y_true, y_obs, pos_label=2)
metrics.auc(fpr, tpr)

In [ ]:
df.write_csv(responses)